# Extending DeepSensor with new models

By default, DeepSensor assumes you are working with convolutional neural process architecture from the `neuralprocesses` library. However, what if you want to use a different model? This may be useful for a number of reasons, such as:
* You want to use a custom model architecture.
* You want to implement benchmarks for ConvNPs, such as Gaussian processes (GPs).
* You don't care about ConvNPs and just want to use the high-level interface of DeepSensor, allowing you to generate `xarray`/`pandas` predictions directly from `xarray`/`pandas` data and perform active learning.

Thankfully, you can easily extend DeepSensor to work with any model you like, as long as you wrap it in a subclass of `DeepSensorModel`!

The design principles enabling this useful extensible behaviour are:
* A base class `ProbabilisticModel` which defines a generic low-level interface for various kinds of predictions that may be needed by downstream functionality, such as `.mean`, `.stddev`, `.sample`, `.covariance`, etc. This blueprint class simply raises a `NotImplementedError` for these methods by default, and is intended to be overridden by subclasses if that functionality is needed.
* A base class `DeepSensorModel` which inherits from `ProbabilisticModel` and defines a generic high-level interface which accepts `Task` objects output by a `TaskLoader` and returns `xarray` or `pandas` objects containing predictions.

The `deepsensor.model.convnp.ConvNP` model class is implemented by subclassing `DeepSensorModel` and overriding the methods of `ProbabilisticModel`. Below is an example of how to implement a new model class.

In [53]:
from deepsensor.data.loader import TaskLoader
from deepsensor.data.processor import DataProcessor
from deepsensor.data.task import Task
from deepsensor.model.model import DeepSensorModel

import xarray as xr
import numpy as np

class NewModel(DeepSensorModel):
    """A very naive model that predicts the mean of the first context set with a fixed stddev"""
    def __init__(self, data_processor: DataProcessor, task_loader: TaskLoader):
        super().__init__(data_processor, task_loader)
    def mean(self, task: Task):
        return np.mean(task["Y_c"][0])
    def stddev(self, task: Task):
        return 0.1

## Demo of DeepSensor functionality using new model class

Now that we have our new model which adheres to the `DeepSensorModel` interface, we can use it in the same way as the default `ConvNP` model. Below is a demo of how to use the new model class.

In [54]:
# Load raw data
ds_raw = xr.tutorial.open_dataset("air_temperature")

In [55]:
# Normalise data
data_processor = DataProcessor(x1_name="lat", x1_map=(ds_raw["lat"].min(), ds_raw["lat"].max()), x2_name="lon", x2_map=(ds_raw["lon"].min(), ds_raw["lon"].max()))
ds = data_processor(ds_raw)

In [56]:
# Set up task loader
task_loader = TaskLoader(context=ds, target=ds)

In [57]:
# Set up model
model = NewModel(data_processor, task_loader)

In [58]:
task = task_loader("2014-12-31", 0.1)
mean_ds, std_ds = model.predict(task, X_t=ds_raw)

In [59]:
mean_ds

<xarray.Dataset>
Dimensions:  (time: 1, lat: 25, lon: 53)
Coordinates:
  * time     (time) datetime64[ns] 2014-12-31
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
Data variables:
    air      (time, lat, lon) float32 272.6 272.6 272.6 ... 272.6 272.6 272.6

In [60]:
std_ds

<xarray.Dataset>
Dimensions:  (time: 1, lat: 25, lon: 53)
Coordinates:
  * time     (time) datetime64[ns] 2014-12-31
  * lat      (lat) float32 75.0 72.5 70.0 67.5 65.0 ... 25.0 22.5 20.0 17.5 15.0
  * lon      (lon) float32 200.0 202.5 205.0 207.5 ... 322.5 325.0 327.5 330.0
Data variables:
    air      (time, lat, lon) float32 1.632 1.632 1.632 ... 1.632 1.632 1.632